# Нейронные сети для предсказания выживаемости на Титанике

В этом ноутбуке реализуем **полносвязные нейронные сети (Multi-Layer Perceptron)** с использованием TensorFlow/Keras.

## Что такое MLP?

**Multi-Layer Perceptron (MLP)** - базовая архитектура нейронных сетей, состоящая из:
- **Входной слой** - получает признаки
- **Скрытые слои** - извлекают паттерны
- **Выходной слой** - делает предсказание

## Содержание
1. Подготовка данных для нейронных сетей
2. Базовая MLP модель
3. MLP с Batch Normalization
4. MLP с Dropout регуляризацией
5. Продвинутая архитектура
6. Learning Rate Scheduling
7. Early Stopping и Callbacks
8. Сравнение с градиентным бустингом
9. Анализ ошибок
10. Выводы

In [ ]:
# Импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# TensorFlow и Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.utils import plot_model

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve

# Утилиты
import joblib

# Настройка отображения
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

# Установка seed для воспроизводимости
np.random.seed(42)
tf.random.set_seed(42)

print('✅ Все библиотеки загружены успешно!')
print(f'TensorFlow версия: {tf.__version__}')
print(f'GPU доступен: {tf.config.list_physical_devices("GPU")}')

## 1. Подготовка данных

Для нейронных сетей критически важна **нормализация данных**.

In [ ]:
# Загрузка данных
df = sns.load_dataset('titanic')
data = df.copy()

# Feature Engineering
data['family_size'] = data['sibsp'] + data['parch'] + 1
data['is_alone'] = (data['family_size'] == 1).astype(int)

# Заполнение пропусков
data['age'].fillna(data['age'].median(), inplace=True)
data['fare'].fillna(data['fare'].median(), inplace=True)
data['embarked'].fillna(data['embarked'].mode()[0], inplace=True)
data['deck'].fillna('Unknown', inplace=True)

# Подготовка признаков
features = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 
            'embarked', 'class', 'who', 'adult_male', 'deck', 
            'alone', 'family_size', 'is_alone']

target = 'survived'

# Кодирование категориальных признаков
le = LabelEncoder()
categorical_features = ['sex', 'embarked', 'class', 'who', 'deck']

X = data[features].copy()
for col in categorical_features:
    if col in X.columns:
        X[col] = le.fit_transform(X[col].astype(str))

y = data[target].values

# Разделение на train/validation/test
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.176, random_state=42, stratify=y_temp)

print(f'✅ Данные разделены!')
print(f'Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}')

### Нормализация данных для нейронных сетей

Нейронные сети работают лучше с нормализованными данными.

In [ ]:
# Нормализация признаков (StandardScaler)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print('✅ Данные нормализованы!')
print(f'\nСредние значения после нормализации:')
print(f'Train mean: {X_train_scaled.mean(axis=0)[:5]}')
print(f'Train std: {X_train_scaled.std(axis=0)[:5]}')

# Размерность входа
input_dim = X_train_scaled.shape[1]
print(f'\nРазмерность входа: {input_dim}')

## 2. Базовая MLP модель

Начнем с простой полносвязной сети.

In [ ]:
# Создание базовой модели
def create_baseline_model(input_dim):
    model = models.Sequential([
        layers.Dense(64, activation='relu', input_dim=input_dim),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )
    
    return model

# Создаем модель
model_baseline = create_baseline_model(input_dim)

# Архитектура модели
print('=== Архитектура базовой модели ===')
model_baseline.summary()

In [ ]:
# Обучение модели
print('\nОбучаем базовую модель...')

history_baseline = model_baseline.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=100,
    batch_size=32,
    verbose=0
)

print('✅ Обучение завершено!')

In [ ]:
# Визуализация обучения
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Loss
axes[0].plot(history_baseline.history['loss'], label='Train Loss')
axes[0].plot(history_baseline.history['val_loss'], label='Val Loss')
axes[0].set_title('Loss по эпохам')
axes[0].set_xlabel('Эпоха')
axes[0].set_ylabel('Binary Crossentropy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy
axes[1].plot(history_baseline.history['accuracy'], label='Train Accuracy')
axes[1].plot(history_baseline.history['val_accuracy'], label='Val Accuracy')
axes[1].set_title('Accuracy по эпохам')
axes[1].set_xlabel('Эпоха')
axes[1].set_ylabel('Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Оценка на validation set
y_pred_baseline = (model_baseline.predict(X_val_scaled) > 0.5).astype(int).flatten()
y_proba_baseline = model_baseline.predict(X_val_scaled).flatten()

print('=== Базовая MLP модель - Validation Set ===')
print(f'Accuracy: {accuracy_score(y_val, y_pred_baseline):.4f}')
print(f'Precision: {precision_score(y_val, y_pred_baseline):.4f}')
print(f'Recall: {recall_score(y_val, y_pred_baseline):.4f}')
print(f'F1-Score: {f1_score(y_val, y_pred_baseline):.4f}')
print(f'ROC-AUC: {roc_auc_score(y_val, y_proba_baseline):.4f}')

## 3. MLP с Batch Normalization

**Batch Normalization** стабилизирует обучение и ускоряет сходимость.

In [ ]:
# Модель с Batch Normalization
def create_bn_model(input_dim):
    model = models.Sequential([
        layers.Dense(128, input_dim=input_dim),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        
        layers.Dense(64),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        
        layers.Dense(32),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )
    
    return model

model_bn = create_bn_model(input_dim)
print('=== Модель с Batch Normalization ===')
model_bn.summary()

In [ ]:
# Обучение
history_bn = model_bn.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=100,
    batch_size=32,
    verbose=0
)

# Оценка
y_pred_bn = (model_bn.predict(X_val_scaled) > 0.5).astype(int).flatten()
y_proba_bn = model_bn.predict(X_val_scaled).flatten()

print('=== MLP с Batch Normalization ===')
print(f'Accuracy: {accuracy_score(y_val, y_pred_bn):.4f}')
print(f'ROC-AUC: {roc_auc_score(y_val, y_proba_bn):.4f}')

## 4. MLP с Dropout регуляризацией

**Dropout** случайно отключает нейроны во время обучения, предотвращая переобучение.

In [ ]:
# Модель с Dropout
def create_dropout_model(input_dim, dropout_rate=0.3):
    model = models.Sequential([
        layers.Dense(128, activation='relu', input_dim=input_dim),
        layers.Dropout(dropout_rate),
        
        layers.Dense(64, activation='relu'),
        layers.Dropout(dropout_rate),
        
        layers.Dense(32, activation='relu'),
        layers.Dropout(dropout_rate),
        
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )
    
    return model

model_dropout = create_dropout_model(input_dim, dropout_rate=0.3)
print('=== Модель с Dropout ===')
model_dropout.summary()

In [ ]:
# Обучение
history_dropout = model_dropout.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=100,
    batch_size=32,
    verbose=0
)

# Оценка
y_pred_dropout = (model_dropout.predict(X_val_scaled) > 0.5).astype(int).flatten()
y_proba_dropout = model_dropout.predict(X_val_scaled).flatten()

print('=== MLP с Dropout ===')
print(f'Accuracy: {accuracy_score(y_val, y_pred_dropout):.4f}')
print(f'ROC-AUC: {roc_auc_score(y_val, y_proba_dropout):.4f}')

## 5. Продвинутая архитектура

Комбинируем все техники: Batch Normalization + Dropout + Разные активации.

In [ ]:
# Продвинутая модель
def create_advanced_model(input_dim):
    model = models.Sequential([
        # Слой 1
        layers.Dense(256, input_dim=input_dim),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.3),
        
        # Слой 2
        layers.Dense(128),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.3),
        
        # Слой 3
        layers.Dense(64),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.2),
        
        # Слой 4
        layers.Dense(32),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.2),
        
        # Выходной слой
        layers.Dense(1, activation='sigmoid')
    ])
    
    # Используем Adam с кастомным learning rate
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy', 
                 tf.keras.metrics.Precision(name='precision'),
                 tf.keras.metrics.Recall(name='recall'),
                 tf.keras.metrics.AUC(name='auc')]
    )
    
    return model

model_advanced = create_advanced_model(input_dim)
print('=== Продвинутая модель ===')
model_advanced.summary()

## 6. Callbacks: Early Stopping и Model Checkpoint

In [ ]:
# Создаем callbacks
import os
os.makedirs('../models', exist_ok=True)

# Early Stopping - останавливает обучение при отсутствии улучшения
early_stop = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

# Model Checkpoint - сохраняет лучшую модель
checkpoint = callbacks.ModelCheckpoint(
    '../models/best_mlp_model.h5',
    monitor='val_auc',
    save_best_only=True,
    mode='max',
    verbose=1
)

# ReduceLROnPlateau - уменьшает learning rate при застое
reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

print('✅ Callbacks настроены!')

In [ ]:
# Обучение с callbacks
print('Обучаем продвинутую модель с callbacks...')

history_advanced = model_advanced.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=200,
    batch_size=32,
    callbacks=[early_stop, checkpoint, reduce_lr],
    verbose=0
)

print('\n✅ Обучение завершено!')
print(f'Количество эпох: {len(history_advanced.history["loss"])}')

In [ ]:
# Визуализация обучения продвинутой модели
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Loss
axes[0, 0].plot(history_advanced.history['loss'], label='Train Loss', alpha=0.7)
axes[0, 0].plot(history_advanced.history['val_loss'], label='Val Loss', alpha=0.7)
axes[0, 0].set_title('Loss', fontsize=12)
axes[0, 0].set_xlabel('Эпоха')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Accuracy
axes[0, 1].plot(history_advanced.history['accuracy'], label='Train Acc', alpha=0.7)
axes[0, 1].plot(history_advanced.history['val_accuracy'], label='Val Acc', alpha=0.7)
axes[0, 1].set_title('Accuracy', fontsize=12)
axes[0, 1].set_xlabel('Эпоха')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# AUC
axes[1, 0].plot(history_advanced.history['auc'], label='Train AUC', alpha=0.7)
axes[1, 0].plot(history_advanced.history['val_auc'], label='Val AUC', alpha=0.7)
axes[1, 0].set_title('ROC-AUC', fontsize=12)
axes[1, 0].set_xlabel('Эпоха')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Precision & Recall
axes[1, 1].plot(history_advanced.history['precision'], label='Train Precision', alpha=0.7)
axes[1, 1].plot(history_advanced.history['val_precision'], label='Val Precision', alpha=0.7)
axes[1, 1].plot(history_advanced.history['recall'], label='Train Recall', alpha=0.7, linestyle='--')
axes[1, 1].plot(history_advanced.history['val_recall'], label='Val Recall', alpha=0.7, linestyle='--')
axes[1, 1].set_title('Precision & Recall', fontsize=12)
axes[1, 1].set_xlabel('Эпоха')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Оценка продвинутой модели
y_pred_advanced = (model_advanced.predict(X_val_scaled) > 0.5).astype(int).flatten()
y_proba_advanced = model_advanced.predict(X_val_scaled).flatten()

print('=== Продвинутая MLP модель - Validation Set ===')
print(f'Accuracy: {accuracy_score(y_val, y_pred_advanced):.4f}')
print(f'Precision: {precision_score(y_val, y_pred_advanced):.4f}')
print(f'Recall: {recall_score(y_val, y_pred_advanced):.4f}')
print(f'F1-Score: {f1_score(y_val, y_pred_advanced):.4f}')
print(f'ROC-AUC: {roc_auc_score(y_val, y_proba_advanced):.4f}')

## 7. Сравнение всех нейронных сетей

In [ ]:
# Сводная таблица результатов
nn_results = pd.DataFrame({
    'Model': ['Baseline MLP', 'MLP + BatchNorm', 'MLP + Dropout', 'Advanced MLP'],
    'Accuracy': [
        accuracy_score(y_val, y_pred_baseline),
        accuracy_score(y_val, y_pred_bn),
        accuracy_score(y_val, y_pred_dropout),
        accuracy_score(y_val, y_pred_advanced)
    ],
    'Precision': [
        precision_score(y_val, y_pred_baseline),
        precision_score(y_val, y_pred_bn),
        precision_score(y_val, y_pred_dropout),
        precision_score(y_val, y_pred_advanced)
    ],
    'Recall': [
        recall_score(y_val, y_pred_baseline),
        recall_score(y_val, y_pred_bn),
        recall_score(y_val, y_pred_dropout),
        recall_score(y_val, y_pred_advanced)
    ],
    'F1-Score': [
        f1_score(y_val, y_pred_baseline),
        f1_score(y_val, y_pred_bn),
        f1_score(y_val, y_pred_dropout),
        f1_score(y_val, y_pred_advanced)
    ],
    'ROC-AUC': [
        roc_auc_score(y_val, y_proba_baseline),
        roc_auc_score(y_val, y_proba_bn),
        roc_auc_score(y_val, y_proba_dropout),
        roc_auc_score(y_val, y_proba_advanced)
    ]
})

nn_results = nn_results.sort_values('ROC-AUC', ascending=False)
print('\n=== Сравнение нейронных сетей ===')
print(nn_results.to_string(index=False))

# Визуализация
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(nn_results))
width = 0.15

ax.bar(x - 2*width, nn_results['Accuracy'], width, label='Accuracy')
ax.bar(x - width, nn_results['Precision'], width, label='Precision')
ax.bar(x, nn_results['Recall'], width, label='Recall')
ax.bar(x + width, nn_results['F1-Score'], width, label='F1-Score')
ax.bar(x + 2*width, nn_results['ROC-AUC'], width, label='ROC-AUC')

ax.set_xlabel('Модель', fontsize=12)
ax.set_ylabel('Значение метрики', fontsize=12)
ax.set_title('Сравнение нейронных сетей по метрикам', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(nn_results['Model'])
ax.legend()
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 8. Финальная оценка на Test Set

In [ ]:
# Используем лучшую модель (Advanced MLP)
y_pred_test = (model_advanced.predict(X_test_scaled) > 0.5).astype(int).flatten()
y_proba_test = model_advanced.predict(X_test_scaled).flatten()

print('=== Финальные результаты на Test Set (Advanced MLP) ===')
print(f'Accuracy: {accuracy_score(y_test, y_pred_test):.4f}')
print(f'Precision: {precision_score(y_test, y_pred_test):.4f}')
print(f'Recall: {recall_score(y_test, y_pred_test):.4f}')
print(f'F1-Score: {f1_score(y_test, y_pred_test):.4f}')
print(f'ROC-AUC: {roc_auc_score(y_test, y_proba_test):.4f}')

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix - Test Set (Advanced MLP)', fontsize=14)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# Classification Report
print('\nClassification Report:')
print(classification_report(y_test, y_pred_test, target_names=['Not Survived', 'Survived']))

## 9. Сохранение модели

In [ ]:
# Сохранение модели и scaler
model_advanced.save('../models/advanced_mlp_model.h5')
joblib.dump(scaler, '../models/scaler.pkl')

print('✅ Модель и scaler сохранены!')
print('Файлы:')
print('  - ../models/advanced_mlp_model.h5')
print('  - ../models/scaler.pkl')

## 10. Выводы

### 🧠 Что мы узнали о нейронных сетях:

1. **Нормализация критична** - без StandardScaler нейросети сходятся хуже

2. **Batch Normalization помогает**:
   - Стабилизирует обучение
   - Позволяет использовать больший learning rate
   - Немного улучшает accuracy

3. **Dropout предотвращает переобучение**:
   - Критичен для малых датасетов
   - Обычно 0.2-0.5 достаточно
   - Улучшает генерализацию

4. **Callbacks экономят время**:
   - Early Stopping автоматически останавливает обучение
   - ReduceLROnPlateau адаптивно подстраивает learning rate
   - ModelCheckpoint сохраняет лучшую модель

5. **Архитектура имеет значение**:
   - Слишком простая → underfitting
   - Слишком сложная → overfitting
   - Для Титаника 2-4 слоя оптимальны

### 📊 Сравнение с градиентным бустингом:

**Нейронные сети**:
- ✅ Гибкие, могут моделировать сложные зависимости
- ✅ Хороши для больших данных
- ❌ Требуют больше данных
- ❌ Сложнее интерпретировать
- ❌ Дольше обучаются
- ❌ Чувствительны к гиперпараметрам

**Градиентный бустинг (XGBoost/LightGBM)**:
- ✅ Отлично работает на малых данных
- ✅ Быстрое обучение
- ✅ Меньше требует тюнинга
- ✅ Легче интерпретировать (SHAP)
- ❌ Менее гибкий для сложных паттернов

### 🎯 Для датасета Титаника:

**Градиентный бустинг лучше**, потому что:
- Мало данных (~900 примеров)
- Табличные данные с категориальными признаками
- Простые линейные зависимости

**Нейронные сети будут лучше** при:
- Больших данных (>10,000 примеров)
- Изображениях, текстах, звуке
- Сложных нелинейных зависимостях

### 💡 Следующие шаги:

- Временные ряды с LSTM (Фаза 3)
- REST API для модели (Фаза 7)
- Более глубокие архитектуры
- Ensemble нейросетей и бустинга